<a href="https://colab.research.google.com/github/NotARectangle/Honours2021/blob/main/InteractDemo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Hello!<br>
This is demo version of the TNGMain model chatbot. <br>
To test the chatbot please run all script parts. At the end it should prompt you for input and the chatbot interaction will start. 
This is only a test, which why the input right now is restricted to just five conversation turns, after the conversation stops. If you want to try again you can run the last step again.

To run all scripts click on Runtime/Laufzeit -> run all/alles ausführen

(Google colab won't install anything on your computer, everything will run in the browser and on the colab servers)

From interact.py file<br> Step 1, install dependencies

In [ ]:
!pip install transformers

     |████████████████████████████████| 2.1MB 5.6MB/s 
     |████████████████████████████████| 901kB 20.6MB/s 
     |████████████████████████████████| 3.3MB 31.0MB/s 


In [ ]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel, top_k_top_p_filtering
import re
from torch.nn import functional as F

Step 2, Dowload the model from the model hub.

In [ ]:
modelPath = "MilaBromm/TNGMain"

model = GPT2LMHeadModel.from_pretrained(modelPath)
tokenizer = GPT2Tokenizer.from_pretrained(modelPath)

Step 3, Prepare the persona Ids

In [ ]:
personaDict = {"PICARD:" :["PICARD: I am Jean-Luc Picard, Captain of the Enterprise. We come in peace.", "PICARD: I am Captain Picard.", "PICARD: I am Jean-Luc Picard, of the Federation Starship, USSEnterprise."],
  "TROI:": ["TROI: I am Troi.","TROI: I'm Deanna Troi, Ship's Counsellor.", "TROI: I'm Deanna.", "TROI: I'm the ship's Counsellor and I thought you might want to talk."],
  "DATA:":["DATA: I am an android. I am Lieutenant Commander Data of the Federation Starship Enterprise.", "DATA: Lieutenant Commander Data.", "DATA: I am Data. An android."],
  "RIKER:":["RIKER: I am Commander William T Riker.", "RIKER: I am Will Riker.", "RIKER: I am First Officer William Riker of the Enterprise."]}

Step 4, prepare chatbot methods

In [ ]:
#This method is responsible for generating the output of the chatbot.
def generate_output(history, persona):
    sequence = history + persona

    for i in range(150):

        input_ids = tokenizer.encode(sequence, return_tensors='pt')

        # get logits of last hidden state
        next_token_logits = model(input_ids).logits[:, -1, :]

        # filter
        filtered_next_token_logits = top_k_top_p_filtering(next_token_logits, top_k=50, top_p=0.95)

        # sample
        probs = F.softmax(filtered_next_token_logits, dim=-1)
        next_token = torch.multinomial(probs, num_samples=1)
        n_token = next_token.tolist()[0][0]
        bos_token_id = tokenizer.bos_token_id
        if n_token == tokenizer.pad_token_id or n_token == bos_token_id:
            continue

        generated = torch.cat([input_ids, next_token], dim=-1)

        resulting_string = tokenizer.decode(generated.tolist()[0])

        #last_token = tokenizer.decode(next_token[0])

        sequence = resulting_string
        last_word = sequence[sequence.rindex(" ") + 1:]

        if re.match("([A-Z]+:)", last_word) or last_word in tokenizer.eos_token:
            resulting_string = resulting_string[: sequence.rindex(last_word)]
            break;

    #just generated outcome
    resulting_string = resulting_string[resulting_string.rindex(persona):]
    return resulting_string

In [ ]:
#loads persona from dictionary
def loadPersona():
    selected =""

    personaIDs = personaDict

    personaKeys = personaIDs.keys()
    personas = ""
    for key in personaKeys:
        personas = personas + key[:-1] + " "
    persNum = len(personaKeys)
    PersonaMsg = "Please select a Persona. personas are " + personas+ "\nPlease type persona selected"
    valid = False
    while not valid:
        print(PersonaMsg)
        userinput = input()
        if userinput.lower() in personas.lower() and len(userinput) > 1:
            idx = personas.index(userinput.upper())
            selected = personas[idx: personas.index(" ", idx)] + ":"
            valid = True
        else:
            print("Not valid. Try again, personas are : " + personas)

    return selected, personaIDs[selected]

In [ ]:
def startChat():
    #select personas
    print("Select Conversation partner")
    persona1, personaID = loadPersona()
    print("Select own persona")
    persona2, personaID2 = loadPersona()
    #start chat.
    history = ["<bos" + " ".join(personaID)] # bos at start of history

    for i in range(5):

        userinput = input(persona2 + " ")

        history.append(persona2 + " " + userinput)
        context = " ".join(history)
        output = generate_output(context, persona1)
        history.append(output)
        print(output)

Last Step, start the chat, to start the chat again, press run again on this cell. 

In [ ]:
startChat()

Select Conversation partner
Please select a Persona. personas are PICARD TROI DATA RIKER 
Please type persona selected
data
Select own persona
Please select a Persona. personas are PICARD TROI DATA RIKER 
Please type persona selected
pic
PICARD: Commander Data, are you there?
DATA: I am in Engineering. I need more time to prepare. I will need some more readings from the plasma plasma analyser. 
PICARD: It is me Captain Picard, I need you back on the Bridge soon.
DATA: Computer, prepare to disengage. Computer, initiate auto-shuttle power drain. I will inform the other departments of your mission and then I will be off duty. 
PICARD: Data, I need you on the Bridge now, don't go off Duty.
DATA: Please, Doctor, please report to your shuttlecraft. 
PICARD: Data, I am starting to get impatient with you. Report to the Bridge now.
DATA: Commander, I am at the Bridge. I am in Engineering. I am unable to disengage myself. 
PICARD: It's Captain, not Commander. How are you able to be in two places